In [ ]:
def plot_multiple_3d_plots(matrix_list, moments, cols=2, tick_interval=10):
    num_plots = len(matrix_list)
    rows = -(-num_plots // cols)  # Calculate rows needed (ceil division)
    
    fig = plt.figure(figsize=(cols * 8, rows * 6))
    axes = [fig.add_subplot(rows, cols, i+1, projection='3d') for i in range(num_plots)]

    for i, (matrix, moment) in enumerate(zip(matrix_list, moments)):
        ax = axes[i]
        matrix = matrix[:, ::-1]  # Reverse the matrix columns
        matrix = matrix[::-1]  # Reverse the matrix rows for correct orientation
        #y_labels = list(map(str, kwp_set))
        y_labels = kwp_set[::-1]
        #y_labels.reverse()

        #x_labels = list(map(str, kw_set))
        x_labels = kw_set[::-1]
        #x_labels.reverse()

        
        # Create meshgrid for 3D plotting
        X, Y = np.meshgrid(np.arange(matrix.shape[1]), np.arange(matrix.shape[0]))
        Z = matrix

        # Plot the 3D surface
        ax.plot_surface(X, Y, Z, cmap='coolwarm')

        # Customize color bar
        cbar = fig.colorbar(ax.plot_surface(X, Y, Z, cmap='coolwarm'), ax=ax, shrink=0.5, aspect=5)
        cbar.ax.tick_params(labelsize=16)

        # Set titles and labels
        ax.set_title(f'{moment}', fontsize=18, weight='bold')
        ax.set_xlabel("EV: charger kW values", fontsize=14)
        ax.set_ylabel("PV: kWp values", fontsize=14)
        #ax.set_zlabel("Value", fontsize=14)

        # Set x and y ticks every 'tick_interval' points
        ax.set_xticks(np.arange(0, len(x_labels), tick_interval))
        ax.set_xticklabels(x_labels[::tick_interval], fontsize=12)
        ax.set_yticks(np.arange(0, len(y_labels), tick_interval))
        ax.set_yticklabels(y_labels[::tick_interval], fontsize=12)
        
    # Hide unused subplots if any
    for i in range(num_plots, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()  # Adjust spacing between subplots
    plt.show()
#plot_multiple_3d_plots([matrix_mean, matrix_std, matrix_skew, matrix_kurt], ["Mean [kWh]","Standard deviation [kWh]","Skewness [/]", "Kurtosis [/]"], cols=2,tick_interval=5)

In [ ]:
# PDF of yearly consumed energy
yearly_load = []
for i, consumer in enumerate(all_consumer_profiles):
    consumer_flat = consumer.values.flatten()
    load = consumer_flat.sum()
    yearly_load.append(load)
    
yearly_load_pandas = pd.Series(yearly_load)
#ax = yearly_load_pandas.plot.density()
plt.hist(yearly_load_pandas,'auto')
ax = plt.gca()
ax.set_title('Yearly consumed energy (over all consumers)')
ax.set_xlabel('kWh')
# plt.hist(yearly_load,bins='fd',density=True)
# plt.axvline(x = 1000, color = 'g', label = 'Small')

# Small and large consumer based on quantiles (middle values)
yearly_load.sort()
q25 = yearly_load_pandas.quantile(0.25)
q75 = yearly_load_pandas.quantile(0.75)
idx25 = bisect.bisect_left(yearly_load, q25)
idx75 = bisect.bisect_left(yearly_load, q75)

load25 = yearly_load[:25]
load75 = yearly_load[75:]
small = load25[round(len(load25)/2)]
large = load75[round(len(load75)/2)]


In [ ]:
def chargingduration(chargingprofile):
    nb_timeframes = np.count_nonzero(chargingprofile)
    hours_charging = nb_timeframes/4
    return hours_charging

print('Total charging hours: ', chargingduration(chargingprofile1))
print('Average hours per day: ', chargingduration(chargingprofile1)/365)

In [ ]:
charging_scenarios = pd.read_csv('./data/Charging_scenarios.csv')
#display(charging_scenarios)
arrival = charging_scenarios['arrival_time']
departure = charging_scenarios['departure_time']
chargetime = charging_scenarios['charge_time']

ax = arrival.plot.density(label = 'Arrival time')
ax.set_title('Arrival time (1 chargingprofile)')
ax.set_xlabel('Hour')

ax = chargetime.plot.density(label = 'Charge duration')
ax.set_title('Arrival time & Charge duration (1 chargingprofile)')
ax.set_xlabel('Hour')
ax.legend()

In [ ]:
consumer1_flat = pd.Series(consumer1.values.flatten())
consumer2_flat = pd.Series(consumer2.values.flatten())
consumer3_flat = pd.Series(consumer3.values.flatten())

ax=consumer1_flat.plot.density(label = 'Small consumer')
consumer2_flat.plot.density(label = 'Medium consumer')
consumer3_flat.plot.density(label = 'Large consumer')
ax.set_title('15 min consumption (over 1 year)')
ax.set_xlabel('kWh')
ax.legend()

In [ ]:
consumer1_flat = pd.Series(consumer1.values.flatten())
solargen_flat = (charging_profiles_dict[7].values.flatten())
kw = 0
kwp = 7
net_load = pd.Series((consumer_flat*4 + solargen_flat)/4)

ax=consumer1_flat.plot.density(label = 'Small consumer')
# consumer2_flat.plot.density(label = 'Medium consumer')
# consumer3_flat.plot.density(label = 'Large consumer')
net_load.plot.density()
ax.set_title('15 min consumption (over 1 year)')
ax.set_xlabel('kWh')
ax.legend()

print(consumer1_flat.std())
print(net_load.std())

In [ ]:
def metrics_whole_profile(consumer):
    # kwp_set = np.linspace(0, 10, 100) # From 0 to 10 with 100 steps
    # kw_set = np.linspace(0, 23, 100)
    step = 0.1
    kwp_set = np.arange(0, 7+step, step)
    kw_set = np.arange(0, 7+step, step)
    kw_set = [round(elem, 1) for elem in kw_set]
    kwp_set = [round(elem, 1) for elem in kwp_set]

    nb_kwp = len(kwp_set)
    nb_kw = len(kw_set)
    # index =  ['Min', 'Max', 'Mean', 'Std', 'Skew', 'Kurt', 'Var', 'Load', 'q5', 'q95', 'Entropy', 'KL', 'TVD', 'Wasserstein']
    # metrics = pd.DataFrame(index=index, columns=['Matrix'], dtype=object)

    matrix_min = np.zeros((nb_kwp, nb_kw))
    matrix_max = np.zeros((nb_kwp, nb_kw))
    matrix_mean = np.zeros((nb_kwp, nb_kw))
    matrix_std = np.zeros((nb_kwp, nb_kw))
    matrix_skew = np.zeros((nb_kwp, nb_kw))
    matrix_kurt = np.zeros((nb_kwp, nb_kw))
    matrix_var = np.zeros((nb_kwp, nb_kw))

    matrix_load = np.zeros((nb_kwp, nb_kw))
    matrix_q5 = np.zeros((nb_kwp, nb_kw))
    matrix_q95 = np.zeros((nb_kwp, nb_kw))

    matrix_entropy = np.zeros((nb_kwp, nb_kw))
    matrix_KL = np.zeros((nb_kwp, nb_kw))
    matrix_total_distance = np.zeros((nb_kwp, nb_kw))
    matrix_wasserstein_distance = np.zeros((nb_kwp, nb_kw))

    consumer_flat = consumer.values.flatten()
    solargen_flat = solargen.values.flatten()
    #chargingprofile1_flat = chargingprofile1.values.flatten()
    
    for i, kwp in enumerate(kwp_set):
        print('iteration :', i, ' of ', nb_kwp)
        for j, kw in enumerate(kw_set):
            if kw == 0:
                charging_profile = charging_profiles_dict[0.1]
                chargingprofile_flat = charging_profile.values.flatten()
            else: 
                charging_profile = charging_profiles_dict[kw]
                chargingprofile_flat = charging_profile.values.flatten()
            chargingprofile_flat[np.isnan(chargingprofile_flat)] = 0

            #chargingprofile_flat = chargingprofile1.values.flatten()
            net_load = consumer_flat*4 - kwp*solargen_flat + chargingprofile_flat
            net_load = net_load/4
            matrix_min[i,j] = net_load.min()
            matrix_max[i,j] = net_load.max()
            matrix_mean[i,j] = net_load.mean()
            matrix_std[i,j] = net_load.std()
            matrix_skew[i,j] = pd.Series(net_load).skew()
            matrix_kurt[i,j] = pd.Series(net_load).kurt()
            matrix_var[i,j] = pd.Series(net_load).var()

            # matrix_skew[kwp,kw] = skew(net_load)
            # matrix_kurt[kwp,kw] = kurtosis(net_load)

            matrix_load[i,j] = net_load.sum()
            matrix_q5[i,j] = pd.Series(net_load).quantile(.05)
            matrix_q95[i,j] = pd.Series(net_load).quantile(.95)

            # #Compute common bin width
            # hist_base, bin_edges_base = np.histogram(consumer_flat,bins='auto',density=False)
            # bin_width = bin_edges_base[1] - bin_edges_base[0]
            # adjusted_min = consumer_flat.min()
            # while adjusted_min > net_load.min():
            #     adjusted_min -= bin_width
            # adjusted_max = consumer_flat.max()
            # while adjusted_max < net_load.max():
            #     adjusted_max += bin_width

            # bin_edges_net_load = np.arange(adjusted_min, adjusted_max + bin_width, bin_width)
            # bin_edges_net_load = [ round(elem, 5) for elem in bin_edges_net_load ]
            # bin_edges_base = [ round(elem, 5) for elem in bin_edges_base ]

            # hist_net_load, _ = np.histogram(net_load,bins=bin_edges_net_load,density=False)

            # # Compute histograms

            # # hist_net_load, bin_edges_net = np.histogram(net_load,bins=bin_edges_net_load,density=True)
            # # hist_base, bin_edges_base = np.histogram(consumer1,bins=bins,range=(global_min,global_max),density=True)
            
            # # Turn hist into 'PDF' (counts to probabilities)
            # hist_net_load = hist_net_load/hist_net_load.sum()
            # hist_base = hist_base/hist_base.sum()
            
            # #S_max = np.log2(nb_bins)
            # matrix_entropy[i,j] = entropy(hist_net_load,base=2) # /S_max
            # epsilon = 1e-10
            # common_bin_edges = np.union1d(bin_edges_base, bin_edges_net_load)
            # hist_base, bin_edges_base = np.histogram(consumer_flat,bins=common_bin_edges,density=False)
            # hist_net_load, bin_edges_net_load = np.histogram(net_load,bins=common_bin_edges,density=False)

            # hist_net_load = hist_net_load/hist_net_load.sum()
            # hist_base = hist_base/hist_base.sum()
            
            # hist_base = hist_base+epsilon
            # hist_net_load = hist_net_load+epsilon

            # matrix_KL[i,j] = entropy(pk=hist_base, qk=hist_net_load, base=2)

            # #dist = np.linalg.norm(np.array(hist_base) - np.array(hist_net_load))
            # #dist = dist/2
            # tvd = 0.5 * np.sum(np.abs(hist_base - hist_net_load))
            # matrix_total_distance[i,j] = tvd

            # midpoints_net = (bin_edges_net_load[:-1] + bin_edges_net_load[1:]) / 2
            # midpoints_base = (bin_edges_base[:-1] + bin_edges_base[1:]) / 2
            # # midpoints = (common_bin_edges[:-1] + common_bin_edges[1:]) / 2
            # # wasserstein_dist = wasserstein_distance(midpoints, midpoints, u_weights=hist_base, v_weights=hist_net_load)
            # wasserstein_dist = wasserstein_distance(midpoints_base, midpoints_net, u_weights=hist_base, v_weights=hist_net_load)
            
            # matrix_wasserstein_distance[i,j] = wasserstein_dist
       
    # metrics.loc['Min', 'Matrix'] = matrix_min
    # metrics.loc['Max', 'Matrix'] = matrix_max
    # metrics.loc['Mean', 'Matrix'] = matrix_mean 
    # metrics.loc['Std', 'Matrix'] = matrix_std 
    # metrics.loc['Skew', 'Matrix'] = matrix_skew 
    # metrics.loc['Kurt', 'Matrix'] = matrix_kurt 
    # metrics.loc['Var', 'Matrix'] = matrix_var
    # metrics.loc['Load', 'Matrix'] = matrix_load 
    # metrics.loc['q5', 'Matrix'] = matrix_q5 
    # metrics.loc['q95', 'Matrix'] = matrix_q95 
    # metrics.loc['Entropy', 'Matrix'] = matrix_entropy 
    # metrics.loc['KL', 'Matrix'] = matrix_KL 
    # metrics.loc['TVD', 'Matrix'] = matrix_total_distance 
    # metrics.loc['Wasserstein', 'Matrix'] = matrix_wasserstein_distance 
    metrics = {}

    metrics["Min"] = matrix_min
    metrics['Max'] = matrix_max
    metrics['Mean'] = matrix_mean 
    metrics['Std'] = matrix_std 
    metrics['Skew'] = matrix_skew 
    metrics['Kurt'] = matrix_kurt 
    metrics['Var'] = matrix_var
    metrics['Load'] = matrix_load 
    metrics['q5'] = matrix_q5 
    metrics['q95'] = matrix_q95
     
    # metrics['Entropy'] = matrix_entropy 
    # metrics['KL'] = matrix_KL 
    # metrics['TVD'] = matrix_total_distance 
    # metrics['Wasserstein'] = matrix_wasserstein_distance 
    return metrics

In [ ]:
print('CONSUMER1 started...')
metrics_consumer_1 = metrics_whole_profile(consumer1)
print('CONSUMER1 done!')

print('CONSUMER2 started...')
metrics_consumer_2 = metrics_whole_profile(consumer2)
print('CONSUMER2 done!')

print('CONSUMER3 started...')
metrics_consumer_3 = metrics_whole_profile(consumer3)
print('CONSUMER3 done!')

In [ ]:
def metrics(consumer):
    step = 0.1
    kwp_set = np.arange(0, 7+step, step)
    kw_set = np.arange(0, 7+step, step)
    kw_set = [round(elem, 1) for elem in kw_set]
    kwp_set = [round(elem, 1) for elem in kwp_set]

    nb_kwp = len(kwp_set)
    nb_kw = len(kw_set)

    # matrix_min = np.zeros((nb_kwp, nb_kw))
    # matrix_max = np.zeros((nb_kwp, nb_kw))
    # matrix_mean = np.zeros((nb_kwp, nb_kw))
    # matrix_std = np.zeros((nb_kwp, nb_kw))
    # matrix_skew = np.zeros((nb_kwp, nb_kw))
    # matrix_kurt = np.zeros((nb_kwp, nb_kw))
    # matrix_var = np.zeros((nb_kwp, nb_kw))
    # matrix_cov = np.zeros((nb_kwp, nb_kw))

    # matrix_load = np.zeros((nb_kwp, nb_kw))
    # matrix_q5 = np.zeros((nb_kwp, nb_kw))
    # matrix_q95 = np.zeros((nb_kwp, nb_kw))

    matrix_entropy = np.zeros((nb_kwp, nb_kw))
    matrix_KL = np.zeros((nb_kwp, nb_kw))
    matrix_total_distance = np.zeros((nb_kwp, nb_kw))
    matrix_wasserstein_distance = np.zeros((nb_kwp, nb_kw))

    # Precompute histograms for consumer[day] once to avoid recomputation
    consumer_histograms = {}
    consumer_bin_edges = {}

    for day in range(1, 366):
        hist_base, bin_edges_base = np.histogram(consumer[day], bins='fd', density=True)
        consumer_histograms[day] = hist_base
        consumer_bin_edges[day] = bin_edges_base

    for i, kwp in enumerate(kwp_set):
        print('iteration :', i, ' of ', nb_kwp)
        for j, kw in enumerate(kw_set):
            print(j, ' kw')
            chargingprofile = charging_profiles_dict[kw]
            chargingprofile = chargingprofile.fillna(0)
            net_load = consumer*4 - kwp*solargen + chargingprofile
            net_load = net_load/4

            # matrix_min[i,j] = net_load.min().min()
            # matrix_max[i,j] = net_load.max().max()
            # matrix_mean[i,j] = net_load.mean().mean()
            # matrix_load[i,j] = net_load.sum().sum()
            # matrix_q5[i,j] = pd.Series(net_load.values.flatten()).quantile(.05)
            # matrix_q95[i,j] = pd.Series(net_load.values.flatten()).quantile(.95)
        
            # matrix_std[i,j] = (net_load.std(axis=0,ddof=0)).sum()/365
            # matrix_skew[i,j] = (net_load.skew(axis=0)).sum()/365
            # matrix_kurt[i,j] = (net_load.kurt(axis=0)).sum()/365
            # matrix_var[i,j] = (net_load.var(axis=0)).sum()/365
            # matrix_cov[i,j] = (net_load.std(axis=0,ddof=0)/net_load.mean(axis=0)).sum()/365

            #Compute common bin width
            # shannon_over_days = [] # Take mean of this
            # kld_over_days = []
            # tvd_over_days = []
            # wasserstein_over_days = []
            shannon_over_days = np.zeros(365)
            kld_over_days = np.zeros(365)
            tvd_over_days = np.zeros(365)
            wasserstein_over_days = np.zeros(365)
            for day in range(1,366):
                net_load_day = net_load[day]
                #hist_base, bin_edges_base = np.histogram(consumer[day],bins='fd',density=True)
                hist_base = consumer_histograms[day]
                bin_edges = consumer_bin_edges[day]

                bin_width = bin_edges_base[1] - bin_edges_base[0]
                adjusted_min = consumer[day].min()
                while adjusted_min > net_load_day.min():
                    adjusted_min -= bin_width
                adjusted_max = consumer[day].max()
                while adjusted_max < net_load_day.max():
                    adjusted_max += bin_width

                bin_edges_net_load_day = np.arange(adjusted_min, adjusted_max + bin_width, bin_width)
                bin_edges_net_load_day = [ round(elem, 5) for elem in bin_edges_net_load_day ]
                bin_edges_base = [ round(elem, 5) for elem in bin_edges_base ]
                hist_net_load_day, _ = np.histogram(net_load_day,bins=bin_edges_net_load_day,density=True)

                # Compute histograms                
                # Turn hist into 'PDF' (counts to probabilities)
                # hist_net_load_day = hist_net_load_day/hist_net_load_day.sum()
                # hist_base = hist_base/hist_base.sum()
                
                #S_max = np.log2(nb_bins)
                # matrix_entropy[i,j] = entropy(hist_net_load_day,base=2) # /S_max
                shannon_over_days[day-1] = entropy(hist_net_load_day,base=2)

                ### KLD
                epsilon = 1e-10
                common_bin_edges = np.union1d(bin_edges_base, bin_edges_net_load_day)
                hist_base, bin_edges_base = np.histogram(consumer[day],bins=common_bin_edges,density=True)
                hist_net_load_day, bin_edges_net_load_day = np.histogram(net_load_day,bins=common_bin_edges,density=True)

                # hist_net_load_day = hist_net_load_day/hist_net_load_day.sum()
                # hist_base = hist_base/hist_base.sum()
                
                hist_base = hist_base+epsilon
                hist_net_load_day = hist_net_load_day+epsilon

                kld_over_days[day-1] = entropy(pk=hist_base, qk=hist_net_load_day, base=2)

                ### TVD
                #dist = np.linalg.norm(np.array(hist_base) - np.array(hist_net_load_day))
                #dist = dist/2
                tvd = 0.5 * np.sum(np.abs(hist_base - hist_net_load_day))
                tvd_over_days[day-1] = tvd

                ### Wasserstein
                midpoints_net = (bin_edges_net_load_day[:-1] + bin_edges_net_load_day[1:]) / 2
                midpoints_base = (bin_edges_base[:-1] + bin_edges_base[1:]) / 2
                # midpoints = (common_bin_edges[:-1] + common_bin_edges[1:]) / 2
                # wasserstein_dist = wasserstein_distance(midpoints, midpoints, u_weights=hist_base, v_weights=hist_net_load)
                wasserstein_dist = wasserstein_distance(midpoints_base, midpoints_net, u_weights=hist_base, v_weights=hist_net_load_day)
                wasserstein_over_days[day-1] = wasserstein_dist

            matrix_entropy[i,j] = np.mean(shannon_over_days)
            matrix_KL[i,j] = np.mean(kld_over_days)
            matrix_total_distance[i,j] = np.mean(tvd_over_days)
            matrix_wasserstein_distance[i,j] = np.mean(wasserstein_over_days)


    metrics = {}
    # metrics["Min"] = matrix_min
    # metrics['Max'] = matrix_max
    # metrics['Mean'] = matrix_mean 
    # metrics['Std'] = matrix_std 
    # metrics['Skew'] = matrix_skew 
    # metrics['Kurt'] = matrix_kurt 
    # metrics['Var'] = matrix_var
    # metrics['Load'] = matrix_load 
    # metrics['q5'] = matrix_q5 
    # metrics['q95'] = matrix_q95
    # metrics['cov'] = matrix_cov
     
    metrics['Entropy'] = matrix_entropy 
    metrics['KL'] = matrix_KL 
    metrics['TVD'] = matrix_total_distance 
    metrics['Wasserstein'] = matrix_wasserstein_distance 
    return metrics